In [42]:
from itertools import combinations
import networkx as nx
import plotly.graph_objs as go

def generate_graphs(k):
    """
    Generate all unique graphs with a total of vertices from 1 to k.
    
    Args:
    k (int): The maximum number of vertices.
    
    Returns:
    list: A list of unique graphs, each represented as a tuple of the number of vertices and a tuple of sorted edges.
    """
    
    all_graphs = {1: [nx.Graph()]}  # Base case: 1 vertex, 0 edges
    
    # Build graphs iteratively from 2 to k vertices
    for num_vertices in range(2, k + 1):
        new_graphs = []
        
        # For each existing graph with (num_vertices - 1)
        for existing_graph in all_graphs[num_vertices - 1]:
            # Create the new graph by adding a new vertex
            new_graph = existing_graph.copy()
            new_graph.add_node(num_vertices - 1)  # Add the new vertex (0-indexed)
            
            # Add the new graph to the list of new graphs (isolated vertex)
            new_graphs.append(new_graph)
            
            # Connect the new vertex to existing vertices in all possible unique ways
            existing_vertices = list(range(num_vertices - 1))
            for r in range(1, num_vertices):
                for combination in combinations(existing_vertices, r):
                    graph_with_edges = new_graph.copy()
                    for vertex in combination:
                        graph_with_edges.add_edge(num_vertices - 1, vertex)
                    new_graphs.append(graph_with_edges)

        # Remove duplicates by checking isomorphism
        unique_graphs = []
        for g in new_graphs:
            if not any(nx.is_isomorphic(g, existing) for existing in unique_graphs):
                unique_graphs.append(g)

        all_graphs[num_vertices] = unique_graphs
    
    # Format the result: convert graphs to tuples of (num_vertices, edges)
    result = []
    for num_vertices in range(1, k + 1):
        for g in all_graphs[num_vertices]:
            edges = tuple(sorted(g.edges()))
            result.append((num_vertices, edges))
    
    return result

def plot_graphs(graphs):
    """
    Plot all generated graphs using Plotly.
    
    Args:
    graphs (list): List of graphs to plot.
    """
    
    for index, (num_vertices, graph_edges) in enumerate(graphs):
        G = nx.Graph()
        G.add_nodes_from(range(num_vertices))
        G.add_edges_from(graph_edges)
        
        # Create a position layout
        if graph_edges:
            pos = nx.spring_layout(G)
        else:
            pos = {i: (i, 0) for i in range(num_vertices)}
        
        # Extracting node positions
        x_nodes = [pos[i][0] for i in G.nodes()]
        y_nodes = [pos[i][1] for i in G.nodes()]

        # Extracting edges for plotting
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x_edge = [pos[edge[0]][0], pos[edge[1]][0], None]
            y_edge = [pos[edge[0]][1], pos[edge[1]][1], None]
            edge_x += x_edge
            edge_y += y_edge
        
        # Create Plotly traces for edges
        edge_trace = go.Scatter(
            x=edge_x, 
            y=edge_y, 
            line=dict(width=0.5, color='gray'),
            hoverinfo='none', 
            mode='lines'
        )
        
        # Create Plotly traces for nodes
        node_trace = go.Scatter(
            x=x_nodes, 
            y=y_nodes,
            mode='markers+text',
            text=[str(i) for i in G.nodes()],
            textposition="middle center",
            hoverinfo='text',
            marker=dict(
                showscale=False,
                size=20,
                color='red',
                line=dict(width=2, color='black')
            ),
            textfont=dict(
                size=16,
                color='white'
            )
        )
        
        # Create the figure
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title=f"Graph with {num_vertices} vertices, {len(graph_edges)} edges",
                            titlefont_size=16,
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0, l=0, r=0, t=40),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                        )
        fig.show()

# Example usage:
k = 4  # Change this value to generate and plot up to k vertices
graphs = generate_graphs(k)
plot_graphs(graphs)